In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import PyPDF2
import os


#### Collect the links on the page

In [2]:
url = "https://www.regjeringen.no/no/dokumenter/horing-nou-2023-25-omstilling-til-lavutslipp-veivalg-for-klimapolitikken-mot-2050-rapport-av-klimautvalget-2050/id3009052/"
response = requests.get(url + "?showSvar=true&term=&page=1&isFilterOpe")

html_doc = response.text

soup = bs(html_doc, 'html.parser')

# Find the ul class that contains the articles
articles = soup.find("ul", class_="link-list")

# Create a dictionary to store the sites and pdfs
sites = {}
pdfs = {}

# Iterate through every li tag in the ul class
for article in articles.find_all("li"):
    if article.text.find("PDF") != -1:
        # Removes the (PDF) from the name of the actor (the writer of the article)
        actor = article.text.split("(")[0][:-1]
        pdfs[actor] = "https://www.regjeringen.no/" + article.find("a")["href"]
        continue
    sites[article.text] = url + article.find("a")["href"]


print(sites)
print(pdfs)


{'Abelia ': 'https://www.regjeringen.no/no/dokumenter/horing-nou-2023-25-omstilling-til-lavutslipp-veivalg-for-klimapolitikken-mot-2050-rapport-av-klimautvalget-2050/id3009052/?uid=c0ad43af-ee84-40ce-bcbe-9288ae5abe33', 'Advokatforeningen ': 'https://www.regjeringen.no/no/dokumenter/horing-nou-2023-25-omstilling-til-lavutslipp-veivalg-for-klimapolitikken-mot-2050-rapport-av-klimautvalget-2050/id3009052/?uid=3dd245ce-31b6-4975-bcc2-702b9da087f8', 'Agder fylkeskommune ': 'https://www.regjeringen.no/no/dokumenter/horing-nou-2023-25-omstilling-til-lavutslipp-veivalg-for-klimapolitikken-mot-2050-rapport-av-klimautvalget-2050/id3009052/?uid=ed6abd47-ef58-4153-974b-5d31b441aa98', 'Akademikerne  ': 'https://www.regjeringen.no/no/dokumenter/horing-nou-2023-25-omstilling-til-lavutslipp-veivalg-for-klimapolitikken-mot-2050-rapport-av-klimautvalget-2050/id3009052/?uid=4b024a54-f322-462d-ac59-8081f66772ab', 'Alliansen ny landbrukspolitikk ': 'https://www.regjeringen.no/no/dokumenter/horing-nou-2023

#### Download pdfs

In [8]:
# Make a directory to store the pdfs
output_dir = "../../dataset/pdf"

os.makedirs(output_dir, exist_ok=True)

# Iterate through the pdfs and download them
for actor, pdf in pdfs.items():
    response = requests.get(pdf)
    output_file = os.path.join(output_dir, actor + ".pdf")
    with open(output_file, "wb") as f:
        f.write(response.content)
        

KeyboardInterrupt: 

#### Read the pdfs and store sections to a csv file

In [21]:

# Preprocessing the pdfs, removing the metadata from the text, and saving the date to the dictionary



def preprocess_pdf(text):
    # Find the beginning of the text
    start_index = text.find("Høring")
    text = text[start_index:]

    return text


# Iterate through the pdfs and extract the text
        
texts = {}

for actor, pdf in pdfs.items():

    pdf_file = os.path.join(output_dir, actor + ".pdf")
    pdf_obj = open(pdf_file, "rb")
    pdf_reader = PyPDF2.PdfReader(pdf_obj)
    num_pages = len(pdf_reader.pages)
    text = ""
    for page_num in range(num_pages):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()
    text = preprocess_pdf(text)
    texts[actor] = text
    # Create a txt file with the text
    print(actor)
    with open(os.path.join("../../dataset/txt_new", actor + ".txt"), "w") as f:
        f.write(text)


Akershus fylkeskommune
Aurskog-Høland kommune
Avfall Norge
Avinor
Barneombudet
Bergen kommune
Den norske kirke
Drivkraft Norge
Elverum kommune
Fagforbundet
Fellesforbundet
Fiskebåt
Grønn Byggallianse
Helsedirektoratet
Jernbanedirektoratet
Kommunesektorens Organisasjon


In [24]:
df_texts = pd.DataFrame(columns=['actor', 'paragraph', 'date'])

# Read the html files and extract the text

def preprocess_html(texts : list):
    # Remove the html tags
    paragraphs = [paragraph.text.strip() for paragraph in texts if paragraph.text != ""]

    # Remove the header "Høringssvar til NOU 2023: ...."
    paragraphs = [sections for sections in paragraphs if sections.find("Høringssvar til NOU 2023") == -1]

    return paragraphs




for actor, site in sites.items():
    print(actor)
    response = requests.get(site)
    html_doc = response.text
    soup = bs(html_doc, 'html.parser')
    # Find the text of the article
    texts = soup.find("div", class_="article-body").find_all("p")
    text = preprocess_html(texts)

    # Find the date of the article
    date = soup.get_text().split("Dato")[1].split("\n")[0].split(" ")[1]


    for paragraph in text:
        df_texts.loc[len(df_texts)] = [actor, paragraph, date]
    


print(actor)
df_texts
df_texts.to_csv("../../dataset/html_hearings.csv", index=False)


Abelia 
Advokatforeningen 
Agder fylkeskommune 
Akademikerne  
Alliansen ny landbrukspolitikk 
Animalia AS 
Ann Norderhaug 
Arbeidsgiverforeningen Spekter 
Arkitektorganisasjonene felles 
Arne Aasen 
Asker kommune 
Askøy kommune 
Atle Guttormsen, Espen Haug og Øystein Sjølie 
Bergen INP 
Bergen kommune 
Besteforeldrenes klimaaksjon 
Besteforeldrenes klimaaksjon, samferdselsgruppa 
Biogass Norge 
Bioteknologirådet 
BirdLife Norge 
Bodø kommune 
Boligprodusentenes Forening 
Buskerud fylkeskommune 
Bærum kommune 
Dag Hendrik Flølo 
Den norske kyrkja ved Biskopen i Møre Ingeborg Midttømme 
Direktoratet for samfunnssikkerhet og beredskap 
DNMF  
Drammen kommune 
Econa 
Eidsvoll INP 
Elkem ASA 
Energigass Norge 
Equinor ASA  
Etne Industri- og næringspartiet 
Eyvind Aven 
Finansforbundet 
Finnmark fylkeskommune  
Fiskeridirektoratet 
FME Norwegian CCS Research Centre/SINTEF Energi 
Forbrukerrådet 
Forbundet for Ledelse og Teknikk 
Fornybar Norge 
Forsvaret, Forsvarsmateriell, Forsvarsbygg og

OSError: Cannot save file into a non-existent directory: '..\..\dataset\csv'

In [26]:
df_texts

,actor,paragraph,date
0,Abelia,Abelia er foreningen for kunnskaps- og teknolo...,30.01.2024
1,Abelia,Innledningsvis vil vi takke for det grundige o...,30.01.2024
2,Abelia,Vi er enig med utvalget i at klimapolitikken m...,30.01.2024
3,Abelia,Over de siste åtte årene har Abelia målt norsk...,30.01.2024
4,Abelia,Vi vil gjerne oppfordre til at det blir satt s...,30.01.2024
...,...,...,...
1394,Lars G Solvik,"Nullutslipps hus? Nei. Det koster både penger,...",28.02.2024
1395,Lars G Solvik,Politikerne må begynne å lytte til sine velger...,28.02.2024
1396,Lars G Solvik,Jeg svarer NEI til NOU: 25,28.02.2024
1397,Larvik kommune,Larvik kommune gir følgende høringsuttalelse t...,27.11.2023
